Import necessary libraries

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [26]:
%matplotlib qt # Use it in order to show plot in a new window -> helpful

UsageError: unrecognized arguments: # Use it in order to show plot in a new window -> helpful


Read data from CSV

In [3]:
df = pd.read_csv('bandusdt.txt')

In [6]:
df.head(2)

,date,timestamp,open,high,low,close,volume,timestamp_c,volume_quote,trades_count,aggresive_buy_volume,aggresive_buy_volume_quote,Ignore,log_ret
0,2022-11-01 00:00:00+00:00,1.667261e+12,1.182,1.183,1.182,1.182,592.0,1.667261e+12,700.0699,7.0,592.0,700.0699,0.0,0.000000
1,2022-11-01 00:01:00+00:00,1.667261e+12,1.182,1.184,1.182,1.184,2987.1,1.667261e+12,3534.2339,25.0,2854.3,3377.0605,0.0,0.001691


Show timestamp in better form

In [9]:
df.timestamp = df.timestamp.astype(np.int64)

In [10]:
df.head(2)

,date,timestamp,open,high,low,close,volume,timestamp_c,volume_quote,trades_count,aggresive_buy_volume,aggresive_buy_volume_quote,Ignore,log_ret
0,2022-11-01 00:00:00+00:00,1667260800000,1.182,1.183,1.182,1.182,592.0,1.667261e+12,700.0699,7.0,592.0,700.0699,0.0,0.000000
1,2022-11-01 00:01:00+00:00,1667260860000,1.182,1.184,1.182,1.184,2987.1,1.667261e+12,3534.2339,25.0,2854.3,3377.0605,0.0,0.001691


Get random timestamps that we will use as a timestamp for our signals

In [22]:
indexes = set(np.random.randint(0, df.shape[0], 50))
timestamps = df.timestamp[indexes].values

Definition of a function for paths creation

In [28]:
# df -> Your candlestick dataframe
# timestamps -> Signal timestamps
# minutes -> How many minutes you would like to plot after the signal

def generate_paths_for_signal_timestamps(df, timestamps, minutes: int=1):
    sigs = {}
    plt.figure()
    plt.title('Paths after signals')
    for t in timestamps:
        temp = df.loc[:, ['close', 'timestamp']] # You can replace close with high/low/open etc.
        temp = temp[(temp.timestamp >= t) & (temp.timestamp <= t + minutes * 1000 * 60)]
        temp.close = np.log(temp.close / temp.close.iloc[0])
        temp.timestamp = temp.timestamp - temp.timestamp.iloc[0]
        if (temp.close.iloc[-1] in list(sigs.values())): # We have this 'if' as an additional protection against the same outcomes from really similar timestamps -> espiecially valuable for HFT data.
            continue
        sigs[t] = temp.close.iloc[-1]
        plt.plot(temp.timestamp / (1000 * 60), temp.close)
    plt.grid()
    plt.show()
    
    return sigs

In [29]:
signals_performance = generate_paths_for_signal_timestamps(df, timestamps, 30)

Stats of your signal performance at the end of the period

In [38]:
np.mean(list(signals_performance.values()))

-0.002216289767372673

In [39]:
np.median(list(signals_performance.values()))

-0.002757196314015331

### ENJOY